In [3]:
import yaml
import requests
import pandas as pd
import json
import sqlalchemy
import pg8000
import pymysql 
from sqlalchemy import Table, Column, Integer, String, MetaData, Date
from sqlalchemy.sql import select, insert

In [1]:
key_file = open('footballYaml.yaml', 'r')
football_key = yaml.safe_load(key_file)['football_key']
db_user = yaml.safe_load(key_file)['db_user']
db_pass = yaml.safe_load(key_file)['db_pass']
db_name = yaml.safe_load(key_file)['db_name']
db_host = yaml.safe_load(key_file)['db_host']


In [5]:
def get_data(year):
    url = "https://v3.football.api-sports.io/fixtures"
    querystring = {"league":"39","season":year}
    payload={}
    headers = {
        'x-rapidapi-key': football_key,
        'x-rapidapi-host': 'v3.football.api-sports.io'
    }
    response = requests.request("GET", url, headers=headers, params=querystring,  data=payload).json()
    season_data = []
    for row in response["response"]:
        season_data.append({"season":row["league"]["season"],
                               "match_date":row["fixture"]["date"],
                               "local_team":row["teams"]["home"]["name"],
                               "away_team":row["teams"]["away"]["name"],
                               "local_goals":row["goals"]["home"],
                               "away_goals":row["goals"]["away"]})
    return season_data


In [6]:
def insert_data(conn, table, data):
    ins = table.insert()
    conn.execute(ins, data)

In [7]:
seasons =[2019, 2020, 2021]
all_season_data = {}

for year in seasons:
    all_season_data[year]= get_data(year)